In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete, ConstantDistribution, \
                          MatrixDistribution
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

In [2]:
%%time
file = './datasets/epsilon_normalized'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))

(n, d) = (400000, 2000)
density = 1.0
CPU times: user 4min 25s, sys: 8.84 s, total: 4min 33s
Wall time: 4min 34s


In [5]:
A = A.toarray()

In [6]:
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A, axis=1))**2 / 4.))

Smoothness based on row norms: 0.2500000724660884


In [31]:
sketch_sizes = [200]
def experiment_epsilon(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=300, 
        mat=A, 
        output_folder='./results/epsilon/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-16, 
        stopping_criteria='grad_abs', 
        random_state=0, 
        methods=['gauss', 'coord', 'bfgs', 'nesterov'], 
        overwrite=False
    )

In [32]:
%%time
experiment_epsilon(1e-4)

RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 33min 15s, sys: 4min 43s, total: 37min 58s
Wall time: 1min 10s


### BFGS

In [29]:
%%time
np.random.seed(42)
oracle = create_log_reg_oracle(A, b, regcoef)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/epsilon/regcoef={:.2e}'.format(regcoef)
method = BFGS(oracle, x_0, stopping_criteria='grad_abs', 
              tolerance=1e-16)
method.run(max_iter=50000, max_time=200)
method.oracle = None
with open('{}/bfgs.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)

CPU times: user 1h 46min 7s, sys: 46.5 s, total: 1h 46min 54s
Wall time: 3min 20s


### Nesterov

In [30]:
%%time
np.random.seed(42)
oracle = create_log_reg_oracle(A, b, regcoef)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/epsilon/regcoef={:.2e}'.format(regcoef)
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=500000, max_time=200)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)

CPU times: user 1h 45min 34s, sys: 1min 24s, total: 1h 46min 59s
Wall time: 3min 21s


### Solve logreg

In [18]:
from methods import LBFGS
from tqdm import tqdm_notebook

In [24]:
def solve_logreg(A, b, regcoef, dataset_name, tolerance=1e-16):
    import os
    import pickle
    os.system('mkdir -p ./logreg_solutions/{}'.format(dataset_name))
    
    oracle = create_log_reg_oracle(A, b, regcoef)
    method = LBFGS(oracle, x_0, tolerance=tolerance, stopping_criteria='grad_abs', 
                   memory_size=10, line_search_options={'method': 'Wolfe'})
    method.run(10000)

    with open('./logreg_solutions/{}/regcoef={:.2e}'\
              .format(dataset_name, regcoef), 'wb') as file:
        pickle.dump({'x_star': method.hist['x_star'], 
                     'func_star': method.hist['func'][-1]}, file)

In [20]:
x_0 = np.zeros((A.shape[1]))

In [25]:
%%time
for regcoef in tqdm_notebook([1e-4]):
    solve_logreg(A, b, regcoef, 'epsilon')


CPU times: user 20min 5s, sys: 9.54 s, total: 20min 14s
Wall time: 38.5 s
